## Исследование надёжности заёмщиков

В данном исследовании установлено, влияет ли семейное положение и количество детей у клиента на вероятность погашения им кредита во время. 

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data= pd.read_csv('/datasets/data.csv')
data.info()
print(data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее 

### Вывод

Существуют пропуски в информации по трудовому стажу и ежемесячному доходу. Также в общем трудовом стаже информация отображается некорректно. Непригодно для работы написаны цели.

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверка на адекватность имеющихся данных  в столбце 'dob_years' : 

In [3]:
data_age_sort=data.sort_values(by = 'dob_years')
print(data_age_sort)

       children  days_employed  dob_years            education  education_id  \
6859          1    -260.992722          0  неоконченное высшее             2   
16042         2    -176.404487          0              среднее             1   
7034          0  366067.781030          0               высшее             0   
21179         2    -108.967042          0               высшее             0   
4147          0   -3549.117333          0              среднее             1   
...         ...            ...        ...                  ...           ...   
3460          0  344623.836105         74              среднее             1   
4895          0  341528.126150         74               высшее             0   
19642         0  380150.387046         74              среднее             1   
11532         0   -6682.867814         74              среднее             1   
8880          0   -1678.969771         75              среднее             1   

               family_status  family_st

Возраст не везде указан корректно. Это может быть связано с нежеланием конкретных людей его указывать по личным причинам. Поскольку эти данные восстановить невозможно, строки нужно удалить. Удаление происходит с учетом того, что минимальный возраст для взятия кредита в РФ - 18 лет.

In [4]:
data=data[data['dob_years']>=18]
data_age_sort=data.sort_values(by = 'dob_years')
print(data_age_sort)

       children  days_employed  dob_years            education  education_id  \
766           0    -796.983636         19  неоконченное высшее             2   
10235         0    -793.358581         19              среднее             1   
13021         0    -695.968951         19  неоконченное высшее             2   
15659         0    -948.225743         19  Неоконченное высшее             2   
4052          0            NaN         19              среднее             1   
...         ...            ...        ...                  ...           ...   
19642         0  380150.387046         74              среднее             1   
4895          0  341528.126150         74               высшее             0   
12317         0   -1729.632531         74              среднее             1   
11532         0   -6682.867814         74              среднее             1   
8880          0   -1678.969771         75              среднее             1   

               family_status  family_st

Проверка чисел в столбце 'children'

In [5]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Предположим, что в столбце есть технические ошбики, а  -1 - это на самом деле 1, а 20 - на самом деле 2.

In [6]:
data['children']=data['children'].replace(-1,1)
data['children']=data['children'].replace(20,2)

In [7]:
print(data['children'].unique())

[1 0 3 2 4 5]


Числа в столбце 'days_employed' прочиворечят здравому смыслу. Это может быть связано с техническими ошибками при хранении или передаче данных.
Работа с 'days_employed':

Прочие проблемы в данных устранены, хорошо.

1)Чтобы легче воспринимались данные из столбца 'days_employed', создается столбец 'years_emploed',  в котором хранится количество проработтаных лет(из расчета, что в году 247 рабочих дней).

In [8]:
data['years_emploed']=data['days_employed']/247
print(data)

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

2)Предмоложим, что отрицательные значения верны по модулю.

In [9]:
data['years_emploed']=abs(data['years_emploed'])
print(data)

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

 3)Поделим данные на возрастные группы и сверим их с трудовым стажем.

In [10]:
def age_group(age):


        if (age <=21)&(age>=18):
                return 'от 18 до 21'
        elif (age>21)&(age<=35):
                return 'от 22 до 35'
        elif (age>35)&(age<=63):    
                return 'от 36 до 63'
        elif (age>63)&(age<90):
                return 'от 63 до 90'
            

In [11]:
data['age_group'] = data['dob_years'].apply(age_group)
data.pivot_table(index='age_group',values='years_emploed',aggfunc=['mean','median'])

,mean,median
,years_emploed,years_emploed
age_group,,
от 18 до 21,2.815983,2.587252
от 22 до 35,10.582988,5.000264
от 36 до 63,313.893086,11.691696
от 63 до 90,1248.274673,1453.673203


Данные во многом не правдоподобные: медианный стаж в 11 лет для людей в возрасте от 36 до 63 подозрителен, средний трудовой стаж - утопичен, как и значения для группы "от 63".

4) Посмотрим на данные с точки зрения трудоустройства.

In [12]:
data.pivot_table(index='income_type',values='years_emploed',aggfunc=['mean','median'])

,mean,median
,years_emploed,years_emploed
income_type,,
безработный,1483.456084,1483.456084
в декрете,13.347206,13.347206
госслужащий,13.718658,10.823502
компаньон,8.553839,6.269788
пенсионер,1477.796468,1478.446708
предприниматель,2.108697,2.108697
сотрудник,9.427545,6.380841
студент,2.343124,2.343124


Данные в столбцах "безработный" и "пенсионер" утопичны.

Предположим, что в столбце 'days_employed' указаны рабочие часы, а не дни. Тогда трудовой стаж в годах сначала делится на 8-мичасовой рабочий день и на количество рабочих дней в году - 247. Тогда расчет стажа работы в годах будет следующим:

Данные в столбцах "безработный" и "пенсионер"  все еще неадекватны.

Если строки, где некорректно представлен трудовой стаж, удалить, то не удастся получить информацию по безработным, пенсионерам, и будет серьезно искажена информация по возрастной группе от 36 до 63. По имеющимся данным восстановить трудовой стаж невозможно. Таким образом, данные в дальнейшем разумней всего не учитывать, а столбец - удалить.

In [14]:
data.drop(['days_employed', 'years_emploed'], axis='columns', inplace=True)
print(data)

       children  dob_years education  education_id     family_status  \
0             1         42    высшее             0   женат / замужем   
1             1         36   среднее             1   женат / замужем   
2             0         33   Среднее             1   женат / замужем   
3             3         32   среднее             1   женат / замужем   
4             0         53   среднее             1  гражданский брак   
...         ...        ...       ...           ...               ...   
21520         1         43   среднее             1  гражданский брак   
21521         0         67   среднее             1   женат / замужем   
21522         1         38   среднее             1  гражданский брак   
21523         3         38   среднее             1   женат / замужем   
21524         2         40   среднее             1   женат / замужем   

       family_status_id gender income_type  debt   total_income  \
0                     0      F   сотрудник     0  253875.639453   
1

Обработка столбца 'total_income'.
В столбце 'total_income' существуют пропуски, это связано с тем, что не все готовы делиться иннформацией относительно своего заработка. Эту информацию можно восполнить через трудоустроенность человека(люди, занятые одним делом зарабатывают примерно одинаково), использовав серднее значение для каждого типа заработка.        

In [16]:
mean=data.pivot_table(index='income_type',values='total_income',aggfunc=['median'])

In [17]:
mean=mean.reset_index()
print(mean)

       income_type         median
                     total_income
0      безработный  131339.751676
1        в декрете   53829.130729
2      госслужащий  150521.917580
3        компаньон  172280.581712
4        пенсионер  118557.179415
5  предприниматель  499163.144947
6        сотрудник  142594.396847
7          студент   98201.625314


Работа со столбцом 'total_income':

NaNы заменяются на нули, затем строки с нулями заполняются средними значениями по типу трудоустройства. 

In [18]:
data['total_income']=data['total_income'].fillna(0)

In [19]:
print(data.loc[(data['income_type'] == 'пенсионер')&(data['total_income']==0),'total_income'].count())

410


In [20]:
for type_gob in data['income_type'].unique():
        data.loc[data['total_income']==0,'total_income'] = data.loc[(data['income_type'] == type_gob)&(data['total_income'] != 0),'total_income'].median()

In [22]:
total_income_sort=data.sort_values(by = 'total_income',ascending= False)
print(total_income_sort)

       children  dob_years education  education_id          family_status  \
12412         0         44    высшее             0        женат / замужем   
19606         1         39    высшее             0        женат / замужем   
9169          1         35   среднее             1       гражданский брак   
20809         0         61   среднее             1  Не женат / не замужем   
17178         0         42    высшее             0       гражданский брак   
...         ...        ...       ...           ...                    ...   
14276         0         61   среднее             1        женат / замужем   
1598          0         68   среднее             1       гражданский брак   
16174         1         52   Среднее             1        женат / замужем   
13006         0         37   среднее             1       гражданский брак   
14585         0         57   среднее             1        женат / замужем   

       family_status_id gender income_type  debt  total_income  \
12412    

Переработаная таблица выглядит следующим образом:

In [23]:
data.pivot_table(index='income_type',values='total_income',aggfunc='median')

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,142594.396847
компаньон,162347.191743
пенсионер,128747.675570
предприниматель,320878.770897
сотрудник,142594.396847
студент,98201.625314


# Вывод

Были убраны строки, где отсутствуют данные по возрасту, столбец 'days_employed', который не пригоден для анализа, восстановлены значения строк для 'total_income' и 'children'. Как следствие, данные стали чище.

### Замена типа данных

Поскольку изначально total_income имеет тип float, для замены типа данных  на целочисленый достаточно вызова метода astype().

In [24]:
data['total_income']=data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
age_group           21424 non-null object
dtypes: int64(6), object(6)
memory usage: 2.1+ MB


### Вывод

Замена типа данных в столбце 'total_income' сделала его более читабельным.

### Обработка дубликатов

Сначала посмотрим, сколько имеется дубликатов на данный момент с помщью метода duplicated():

In [25]:
data.duplicated().sum()

54

Это те дубликаты, которые способен распознать метод. Чтобы найти остальные, нужно привести к единообразию тексты строк, все буквы должны иметь нижний регистр. Для этого просматриваются все уникальные значения в столбцах с типом данных 'object'. Если среди них есть верхний регистр, от него необходимо избавиться. 

In [27]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [28]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Для того, чтобы убрать повторы в 'education', необходимо избавиться от верхних регистров через метод str.lower() . 

In [29]:
print(data['education'].unique()) 
data['education']=data['education'].str.lower()
print(data['education'].unique())   

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [30]:
data.duplicated().sum()

71

Избавившись от верхних регистров, можно удалить все найденные дубликаты через метод duplicated() которых ищет повторы в данных.

In [31]:
data=data.drop_duplicates().reset_index(drop = True)

Проверка результата:

In [32]:
data.duplicated().sum()

0

### Вывод

Количество удаленных дубликатов незначительно и не должно существенно отразиться на качестве требуемых расчетов. Часть дубликатов могла появиться из-за технических неполадок. После смены регистра количество дубликатов увеличилось, можно предположить, что некоторые люди были внесены в базу дважды. 

### Лемматизация

Для лемматизации необходимо вызвать библиотеку  pymystem3

In [33]:
from pymystem3 import Mystem
m = Mystem()
 

Затем создается функция, которая лемматизирует(приводит к словарной форме) вводные данные. Поскольку резульатом метода lemmatize() является список, с помощью метода join() он преобразовывается в строку и в таком виде вносится в столбец 'purpose_lemmas' c помощью метода apply().

In [34]:
def lemmas(row):
    lemmas = ''.join(m.lemmatize(row))
    return lemmas
data['purpose_lemmas']=data['purpose'].apply(lemmas)
print(data)   

       children  dob_years education  education_id     family_status  \
0             1         42    высшее             0   женат / замужем   
1             1         36   среднее             1   женат / замужем   
2             0         33   среднее             1   женат / замужем   
3             3         32   среднее             1   женат / замужем   
4             0         53   среднее             1  гражданский брак   
...         ...        ...       ...           ...               ...   
21348         1         43   среднее             1  гражданский брак   
21349         0         67   среднее             1   женат / замужем   
21350         1         38   среднее             1  гражданский брак   
21351         3         38   среднее             1   женат / замужем   
21352         2         40   среднее             1   женат / замужем   

       family_status_id gender income_type  debt  total_income  \
0                     0      F   сотрудник     0        253875   
1  

### Вывод

Цели были приведены к словарной форме, в таком виде данные пригодны для категоризации.

### Категоризация данных

Необходимо свести леммы к одному слову, предмету покупки. При этом нужно учесть, что цели "жилье" и "недвижимость" являются одним и тем же предметом покупки.

In [74]:
lemmas_common=' '.join(data['purpose'])
#print(lemmas_common)
print(Counter(m.lemmatize(lemmas_common)).most_common(50))

[(' ', 54787), ('недвижимость', 6328), ('покупка', 5870), ('жилье', 4436), ('автомобиль', 4284), ('образование', 3995), ('с', 2904), ('операция', 2593), ('свадьба', 2310), ('свой', 2223), ('на', 2210), ('строительство', 1873), ('высокий', 1366), ('получение', 1309), ('коммерческий', 1306), ('для', 1286), ('жилой', 1224), ('сделка', 938), ('дополнительный', 902), ('заниматься', 900), ('подержать', 849), ('проведение', 764), ('сыграть', 760), ('сдача', 649), ('семья', 637), ('собственный', 633), ('со', 627), ('ремонт', 605), ('приобретение', 459), ('профильный', 435), ('подержанный', 113), ('\n', 1)]


In [37]:
def category(row):
    for word in row.split():
        if (word=='недвижимость')|(word=='жилье'):
            return 'недвижимость'
        if (word=='свадьба'):
            return 'свадьба'
        if (word=='автомобиль'):
            return 'автомобиль'
        if (word=='образование'):
            return 'образование'
data['purpose_сategory']=data['purpose_lemmas'].apply(category)
print(data)

       children  dob_years education  education_id     family_status  \
0             1         42    высшее             0   женат / замужем   
1             1         36   среднее             1   женат / замужем   
2             0         33   среднее             1   женат / замужем   
3             3         32   среднее             1   женат / замужем   
4             0         53   среднее             1  гражданский брак   
...         ...        ...       ...           ...               ...   
21348         1         43   среднее             1  гражданский брак   
21349         0         67   среднее             1   женат / замужем   
21350         1         38   среднее             1  гражданский брак   
21351         3         38   среднее             1   женат / замужем   
21352         2         40   среднее             1   женат / замужем   

       family_status_id gender income_type  debt  total_income  \
0                     0      F   сотрудник     0        253875   
1  

### Вывод

Леммы категоризированы и пригодны для анализа.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Данные группируются по количеству детей и делятся на два столбца: количество тех кто вернул кредит в срок и тех, кто имеет просрочки по кредиту.

In [38]:
data_children_debt=data.pivot_table(index='children',columns='debt',values='total_income',aggfunc='count')
data_children_debt.reset_index(inplace=True)
data_children_debt.columns=['children','debt_false','debt_true']
print(data_children_debt)

   children  debt_false  debt_true
0         0     12964.0     1058.0
1         1      4397.0      442.0
2         2      1912.0      202.0
3         3       301.0       27.0
4         4        37.0        4.0
5         5         9.0        NaN


Строка с 5 детьми удаляется, так как в ней слишком мало данных для анализа( 9 человек). 

In [39]:
data_children_debt.drop(5, inplace=True)
print(data_children_debt)

   children  debt_false  debt_true
0         0     12964.0     1058.0
1         1      4397.0      442.0
2         2      1912.0      202.0
3         3       301.0       27.0
4         4        37.0        4.0


Суммируются оба столбца, чтобы найти сумму заемщиков, затем создаются столбцы с процентами тех, кто вернул и не вернул кредит в срок.

In [40]:
data_children_debt['sum']=data_children_debt['debt_false']+data_children_debt['debt_true']
data_children_debt['pr_debt_false']=data_children_debt['debt_false']/data_children_debt['sum']*100
data_children_debt['pr_debt_true']=data_children_debt['debt_true']/data_children_debt['sum']*100
print(data_children_debt)

   children  debt_false  debt_true      sum  pr_debt_false  pr_debt_true
0         0     12964.0     1058.0  14022.0      92.454714      7.545286
1         1      4397.0      442.0   4839.0      90.865881      9.134119
2         2      1912.0      202.0   2114.0      90.444655      9.555345
3         3       301.0       27.0    328.0      91.768293      8.231707
4         4        37.0        4.0     41.0      90.243902      9.756098


### Вывод

В целом, с увеличением количества детей количество вернувших кредит в срок уменьшается. Однако эти изменения незначительны, количество детей не может стать решающим показателем в принятии решения по выдаче кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Данные группируются по семейному статусу и делятся на два столбца: количество тех кто вернул кредит в срок и тех, кто имеет просрочки по кредиту.

In [41]:
data_family_debt=data.pivot_table(index=['family_status','family_status_id'],columns='debt',values='total_income',aggfunc='count')
data_family_debt.reset_index(inplace=True)
data_family_debt.columns=['family_status','family_status_id','debt_false','debt_true']

Суммируются оба столбца, чтобы найти сумму заемщиков, затем создаются столбцы с процентами тех, кто вернул и не вернул кредит в срок.

In [42]:
data_family_debt['sum']=data_family_debt['debt_false']+data_family_debt['debt_true']
data_family_debt['pr_debt_false']=data_family_debt['debt_false']/data_family_debt['sum']*100
data_family_debt['pr_debt_true']=data_family_debt['debt_true']/data_family_debt['sum']*100
print(data_family_debt)

           family_status  family_status_id  debt_false  debt_true    sum  \
0  Не женат / не замужем                 4        2521        273   2794   
1              в разводе                 3        1100         85   1185   
2         вдовец / вдова                 2         892         62    954   
3       гражданский брак                 1        3744        386   4130   
4        женат / замужем                 0       11363        927  12290   

   pr_debt_false  pr_debt_true  
0      90.229062      9.770938  
1      92.827004      7.172996  
2      93.501048      6.498952  
3      90.653753      9.346247  
4      92.457282      7.542718  


### Вывод

Нет разницы между наличием партнера у занимающего и его отсутствием (чаще всего срывают сроки люди не состоящие в браке и те, что состоят в гражданском браке). Меньше всего просрочек у вдовствующих, что может быть связано сколько не с семейным положением, а с возрастом данной группы людей (вдовствует в большинстве своем пожилая группа людей). Этот показатель также не может быть решающим в принятии решения по выдаче кредита.

Содается столбец 'earn_сategory', в котором уровень дохода делится на "высокий", "средний", "низкий".

In [45]:
def category(row):
        if (row<=quant[0.2]):
            return 'низкая'
        if (row>quant[0.2])&(row<=quant[0.4]):
            return 'ниже среднего'
        if (row>quant[0.4])&(row<=quant[0.6]):
            return 'средняя'
        if (row>quant[0.6])&(row<=quant[0.8]):
            return 'выше среднего'
        if (row>=quant[0.8]):
            return 'высокая'

        
data['earn_сategory']=data['total_income'].apply(category)
data.reset_index()

,index,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemmas,purpose_сategory,earn_сategory
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 36 до 63,покупка жилье\n,недвижимость,высокая
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 36 до 63,приобретение автомобиль\n,автомобиль,ниже среднего
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 22 до 35,покупка жилье\n,недвижимость,средняя
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 22 до 35,дополнительный образование\n,образование,высокая
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,от 36 до 63,сыграть свадьба\n,свадьба,выше среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21348,21348,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,от 36 до 63,операция с жилье\n,недвижимость,высокая
21349,21349,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,от 63 до 90,сделка с автомобиль\n,автомобиль,средняя
21350,21350,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,от 36 до 63,недвижимость\n,недвижимость,низкая
21351,21351,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,от 36 до 63,на покупка свой автомобиль\n,автомобиль,высокая


Данные группируются по уровню дохода и делятся на два столбца: количество тех кто вернул кредит в срок и тех, кто имеет просрочки по кредиту. Суммируются оба столбца, чтобы найти сумму заемщиков, затем создаются столбцы с процентами тех, кто вернул и не вернул кредит в срок.

In [46]:
data_earn_dedt=data.pivot_table(index='earn_сategory',columns='debt',values='total_income',aggfunc='count')
data_earn_dedt.reset_index(inplace=True)
data_earn_dedt.columns=['earn_сategory','debt_false','debt_true']
data_earn_dedt['sum']=data_earn_dedt['debt_false']+data_earn_dedt['debt_true']
data_earn_dedt['pr_debt_false']=data_earn_dedt['debt_false']/data_earn_dedt['sum']*100
data_earn_dedt['pr_debt_true']=data_earn_dedt['debt_true']/data_earn_dedt['sum']*100
print(data_earn_dedt)

   earn_сategory  debt_false  debt_true   sum  pr_debt_false  pr_debt_true
0        высокая        3971        299  4270      92.997658      7.002342
1  выше среднего        3903        368  4271      91.383751      8.616249
2  ниже среднего        3919        352  4271      91.758370      8.241630
3         низкая        3928        343  4271      91.969094      8.030906
4        средняя        3899        371  4270      91.311475      8.688525


### Вывод

Люди с высокой и средней зарплатой чаще имеют просрочки по кредитам, чем люди с низкой зарплатой. При этом, чем выше зарплата, тем кредиты выдаются чаще. Можно предположить, что у банков более высокие требования к людям с низкой зарплатой, им кредиты выдаются менее охотно из-за рисков, поэтому процент возврата в срок самый высокий.

Данные группируются по уровню дохода и делятся на два столбца: количество тех кто вернул кредит в срок и тех, кто имеет просрочки по кредиту.

In [47]:
data_purpose_debt=data.pivot_table(index='purpose_сategory',columns='debt',values='total_income',aggfunc='count')
data_purpose_debt.reset_index(inplace=True)
data_purpose_debt.columns=['purpose_сategory','debt_false','debt_true']
print(data_purpose_debt)

  purpose_сategory  debt_false  debt_true
0       автомобиль        3884        400
1     недвижимость        9985        779
2      образование        3625        370
3          свадьба        2126        184


Суммируются оба столбца, чтобы найти сумму заемщиков, затем создаются столбцы с процентами тех, кто вернул и не вернул кредит в срок.

In [48]:
data_purpose_debt['sum']=data_purpose_debt['debt_false']+data_purpose_debt['debt_true']
data_purpose_debt['pr_debt_false']=data_purpose_debt['debt_false']/data_purpose_debt['sum']*100
data_purpose_debt['pr_debt_true']=data_purpose_debt['debt_true']/data_purpose_debt['sum']*100
print(data_purpose_debt)

  purpose_сategory  debt_false  debt_true    sum  pr_debt_false  pr_debt_true
0       автомобиль        3884        400   4284      90.662932      9.337068
1     недвижимость        9985        779  10764      92.762913      7.237087
2      образование        3625        370   3995      90.738423      9.261577
3          свадьба        2126        184   2310      92.034632      7.965368


### Вывод

 За недвижимость и свадьбу люди возвращают кредит с высокой регулярностью. Можно предположить, что за недвижимость возвращают кредит чаще всего из-за крайней необходимости (за неуплату человек может остаться без жилья, базовой потребности человека). Кредит на свадьбу чаще остальных оплачивается, так как по нему выплаты не такие большие и не такие долгие, как по остальным типам выдачи.    

### Шаг 4. Общий вывод

- Среди рассматриваемых факторов нет преобладающего, по которому можно однозначно принять решение по выдаче кредита, так как различия между полученными значениями варьируются в 1%-3%. 
- Однозначно можно сказать, что чем больше детей, тем чаще люди срывают выплаты по кредитам.
- Остальные результаты требуют дальнейшего рассмотрения:
- - возврат кредита в срок может зависеть сколько не от семейного положения, а от возраста человека.
- - cамый высокий процент возврата кредита в срок среди людей с низкой зарплатой может быть связан с малым количеством заемщиков в этой группе. Больший процент просрочек среди людей с высокой и средней зарплатой может нивелироваться за счет того, что таких кредитов выдается в принципе больше, а подавляющая доля людей таки возвращает их в срок. То есть, нельзя утверждать, что человек с низким уровнем дохода является самым рациональным для выдачи кредита.
- - высокая регулярность возвратов кредитов на свадьбу может бысть связана не с самой целью, а с меньшей по сравнению с другими группами суммой выплат( или длиной выплат).